This notebook is for collecting a cleaning data for the model.

In [207]:
import pandas as pd
from selenium import webdriver
from csv import writer
import datetime
import html5lib
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from urllib.request import urlopen
from lxml import html
import requests
from bs4 import BeautifulSoup, Comment
import re
from unidecode import unidecode

chrome_options = Options()
chrome_options.add_extension('/Users/alexstratton2121/Library/Application Support/Google/Chrome/Default/Extensions/gighmmpiobklfepjocnamgkkbiglidom/5.0.2_0.crx')

In [202]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [ ]:
def ev_calculator(win_prob, stake, win_odds, loss_odds):
    winnings = (100/win_odds)*stake
    loss_prob = 1 - win_prob
    ev = (win_prob*winnings)-(loss_prob*stake)
    
    return ev

1. Get historical lineups from bbal reference
2. Match player's raptor ratings with lineups
3. 

In [230]:
def get_lineups(season):
    if season == '2020':
        months = ["october-2019", "november", "december", "january", 
                  "february", "march", "july", "august", "september", "october-2020"]
    elif season == '2021':
        months = ["december", "january", "february", "march", "april", "may", "june", "july"]
    else:
        months = ["october", "november", "december", "january", "february", "march", "april"]
        
    lineups = []
    
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument('disable-notifications')
    chrome_options.add_argument("window-size=1280,720")
    
    for month in months:
        driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
        wait = WebDriverWait(driver, 10)
        # Not every season starts/ends in the same month due to Covid. Hence the correct months need to be passed.
        driver.get("https://www.basketball-reference.com/leagues/NBA_"+season+"_games-"+month+".html")

        wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@id="all_schedule"]'))).location_once_scrolled_into_view
        games = wait.until(EC.presence_of_element_located((By.XPATH, '//table[@id="schedule"]')))
        # Getting links to the boxscore of each game
        links = [x.get_attribute('href') for x in games.find_elements(By.TAG_NAME, 'a') if x.text == 'Box Score']
            
        for link in links:
            driver.get(link)
            wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="line_score"]/tbody/tr[1]/th/a'))).location_once_scrolled_into_view
            teams = [str(driver.find_element(By.XPATH, '//*[@id="line_score"]/tbody/tr[1]/th/a').text),
                     str(driver.find_element(By.XPATH, '//*[@id="line_score"]/tbody/tr[2]/th/a').text)]
            r = requests.get(link)
            soup = BeautifulSoup(r.text, 'html.parser')
                
            for team in teams:
                # Getting boxscore and adding team_id into each dataframe.
                table = soup.find(id="box-"+str(team)+"-game-basic")
                lineups.append(pd.read_html(table.prettify())[0])
                lineups[len(lineups)-1].iloc[5, 0] = team
    
    # Some players have letters not in the English language in their name. The dataset I need to match this to \
    # does not have these letters. Therefore I have to find a remove them. Example: 'Nikola Vučević'
    for lineup in lineups:
        for player in lineup.iloc[:, 0]:
            if isEnglish(player) == False:
                player = unidecode(player)
                
    return lineups

In [ ]:
lineups = [get_lineups(str(2014 + i)) for i in range(8)]

In [240]:
raptor = pd.read_excel("/Users/alexstratton2121/Algo Betting/raptor_data.xlsx")
raptor = [raptor.loc[raptor['season'] == 2014+i] for i in range(8)]
for year in raptor:
    year = year.loc[year['season_type'] == "RS"]

In [223]:
def match(lineups, raptor):
    raptor_groups = raptor.groupby(by='team')
    
    for key, players in raptor_groups:
        for lineup in lineups:
            if lineup.iloc[5, 0] == key:
                for idx, player in enumerate(lineup.iloc[:, 0]):
                    lineup.iloc[idx, 1] = players.loc[players['player_name'] == player, 'raptor_box_total'].item()
            else:
                continue
    
    
    return lineups

In [224]:
combined = [match(lineups[i], raptor[i]) for i in range(len(lineups))]